In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
import os


In [2]:
classifier = Sequential()

In [3]:
classifier.add(Convolution2D(32, (3, 3), input_shape=(128, 128, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

In [4]:
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))


In [5]:
classifier.add(Flatten())

In [6]:
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=96, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=26, activation='softmax'))

In [7]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
classifier.summary()

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [9]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory('data/train',
                                                 target_size=(128, 128),
                                                 batch_size=1,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('data/test',
                                            target_size=(128 , 128),
                                            batch_size=1,
                                            color_mode='grayscale',
                                            class_mode='categorical') 


Found 7800 images belonging to 26 classes.
Found 1300 images belonging to 26 classes.


In [13]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=7800, 
        epochs=10,
        validation_data=test_set,
        validation_steps=1300)

Epoch 1/10
   7/7800 [..............................] - ETA: 2:10 - loss: 1.0437 - accuracy: 0.5714

C:\Users\aeshi\anaconda3\lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


7800/7800 [==============================] - 153s 20ms/step - loss: 1.0991 - accuracy: 0.6482 - val_loss: 1.8751 - val_accuracy: 0.4669
Epoch 2/10
7800/7800 [==============================] - 160s 21ms/step - loss: 0.7222 - accuracy: 0.7729 - val_loss: 1.9575 - val_accuracy: 0.5115
Epoch 3/10
7800/7800 [==============================] - 177s 23ms/step - loss: 0.6060 - accuracy: 0.8103 - val_loss: 2.5676 - val_accuracy: 0.4408
Epoch 4/10
7800/7800 [==============================] - 193s 25ms/step - loss: 0.5331 - accuracy: 0.8404 - val_loss: 2.9446 - val_accuracy: 0.4877
Epoch 5/10
7800/7800 [==============================] - 175s 22ms/step - loss: 0.4601 - accuracy: 0.8587 - val_loss: 2.3926 - val_accuracy: 0.4923
Epoch 6/10
7800/7800 [==============================] - 172s 22ms/step - loss: 0.4400 - accuracy: 0.8715 - val_loss: 2.6694 - val_accuracy: 0.5015
Epoch 7/10
7800/7800 [==============================] - 201s 26ms/step - loss: 0.3991 - accuracy: 0.8821 - val_loss: 3.1310 - val

In [14]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model-bw.h5')
print('Weights saved')

Model Saved
Weights saved
